In [22]:
# -*- coding: utf-8 -*-

import os
import sys
import csv

import ccxt  # noqa: E402

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   num_retries = 0
   try:
       num_retries += 1
       ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
       # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
       return ohlcv
   except Exception:
       if num_retries > max_retries:
           raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
   timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
   timedelta = limit * timeframe_duration_in_ms
   now = exchange.milliseconds()
   all_ohlcv = []
   fetch_since = since
   while fetch_since < now:
       ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
       fetch_since = (ohlcv[-1][0] + 1) if len(ohlcv) else (fetch_since + timedelta)
       all_ohlcv = all_ohlcv + ohlcv
       if len(all_ohlcv):
           print(len(all_ohlcv), 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
       else:
           print(len(all_ohlcv), 'candles in total from', exchange.iso8601(fetch_since))
   return exchange.filter_by_since_limit(all_ohlcv, since, None, key=0)


def write_to_csv(filename, data):
   with open(filename, mode='w') as output_file:
       csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
       csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
   # instantiate the exchange by id
   exchange = getattr(ccxt, exchange_id)()
   # convert since from string to milliseconds integer if needed
   if isinstance(since, str):
       since = exchange.parse8601(since)
   # preload all markets from the exchange
   exchange.load_markets()
   # fetch all candles
   ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
   # save them to csv file
   write_to_csv(filename, ohlcv)
   print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

# Binance's BTC/USDT candles start on 2017-08-17
scrape_candles_to_csv('ETH_h.csv', 'binance', 3, 'ETH/USDT', '1h', '2021-01-01T00:00:00Z', 100)

100 candles in total from 2021-01-01T00:00:00.000Z to 2021-01-05T03:00:00.000Z
200 candles in total from 2021-01-01T00:00:00.000Z to 2021-01-09T07:00:00.000Z
300 candles in total from 2021-01-01T00:00:00.000Z to 2021-01-13T11:00:00.000Z
400 candles in total from 2021-01-01T00:00:00.000Z to 2021-01-17T15:00:00.000Z
500 candles in total from 2021-01-01T00:00:00.000Z to 2021-01-21T19:00:00.000Z
600 candles in total from 2021-01-01T00:00:00.000Z to 2021-01-25T23:00:00.000Z
700 candles in total from 2021-01-01T00:00:00.000Z to 2021-01-30T03:00:00.000Z
800 candles in total from 2021-01-01T00:00:00.000Z to 2021-02-03T07:00:00.000Z
900 candles in total from 2021-01-01T00:00:00.000Z to 2021-02-07T11:00:00.000Z
1000 candles in total from 2021-01-01T00:00:00.000Z to 2021-02-11T16:00:00.000Z
1100 candles in total from 2021-01-01T00:00:00.000Z to 2021-02-15T20:00:00.000Z
1200 candles in total from 2021-01-01T00:00:00.000Z to 2021-02-20T00:00:00.000Z
1300 candles in total from 2021-01-01T00:00:00.00

In [34]:
import pandas as pd
from datetime import datetime

In [51]:
history = pd.read_csv('ETH_h.csv', header=None, names=['time', 'open', 'high', 'low', 'close', 'volume'])
history.dropna(inplace=True)
history

,time,open,high,low,close,volume
0,1609459200000,736.42,739.00,729.33,734.07,27932.69884
1,1609462800000,734.08,749.00,733.37,748.28,52336.18779
2,1609466400000,748.27,749.00,742.27,744.06,33019.50100
3,1609470000000,744.06,747.23,743.10,744.82,17604.80859
4,1609473600000,744.87,747.09,739.30,742.29,18794.15424
...,...,...,...,...,...,...
28407,1711774800000,3507.41,3532.77,3505.09,3508.01,22373.90000
28408,1711778400000,3508.00,3508.56,3493.48,3499.60,11278.19630
28409,1711782000000,3499.59,3503.00,3492.34,3492.64,8188.97140
28410,1711785600000,3492.65,3510.00,3492.20,3505.24,10240.79110


In [52]:
history['time'] = pd.to_datetime(history['time'], unit='ms')
dt = datetime(2024, 3, 11, 0, 0, 0)
history = history[history['time'] < dt]
history['date'] = history['time'].dt.date
history['hour'] = history['time'].dt.time
history

C:\Users\HP\AppData\Local\Temp\ipykernel_17820\2343537301.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['date'] = history['time'].dt.date
C:\Users\HP\AppData\Local\Temp\ipykernel_17820\2343537301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['hour'] = history['time'].dt.time


,time,open,high,low,close,volume,date,hour
0,2021-01-01 00:00:00,736.42,739.00,729.33,734.07,27932.69884,2021-01-01,00:00:00
1,2021-01-01 01:00:00,734.08,749.00,733.37,748.28,52336.18779,2021-01-01,01:00:00
2,2021-01-01 02:00:00,748.27,749.00,742.27,744.06,33019.50100,2021-01-01,02:00:00
3,2021-01-01 03:00:00,744.06,747.23,743.10,744.82,17604.80859,2021-01-01,03:00:00
4,2021-01-01 04:00:00,744.87,747.09,739.30,742.29,18794.15424,2021-01-01,04:00:00
...,...,...,...,...,...,...,...,...
27941,2024-03-10 19:00:00,3893.31,3902.13,3888.85,3898.00,7832.29130,2024-03-10,19:00:00
27942,2024-03-10 20:00:00,3898.01,3903.89,3892.21,3900.37,5745.05080,2024-03-10,20:00:00
27943,2024-03-10 21:00:00,3900.36,3903.40,3875.00,3880.55,7156.31930,2024-03-10,21:00:00
27944,2024-03-10 22:00:00,3880.54,3882.17,3791.26,3823.70,39770.84150,2024-03-10,22:00:00
